# 1. Lane-Emden Equation and Polytropes

## a. Lane-Emden Equation Integrator

In [1]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np

def euler(n,h):
    
    _x = []
    _theta =[]
    _f = []
    
    theta = 1
    f = 0
    x = 0
   
    k = 0
    while theta >= 0:
        
        x = h*(k+1)
        
        fp = -theta**n - (2./x)*f
        df = h * fp
        f += df
        
        dtheta = h * f
        theta += dtheta
        
        k += 1
        
        _theta.append(theta)
        _x.append(h*k)
        _f.append(f)
        
    return np.array(_x), np.array(_theta), np.array(_f)

def rk4(n,h,disp = False,ndisp=100):
    """
    Follow the notation and recipe from HWT chapter 7:
        y = theta_n
        x = xi
        z = dy/dx
        
    """
    
    _x = []
    _y = []
    _z = []
    
    
    # Start the integration just off the origin and get y and y' from the taylor
    # expansion for y
    
    x0 = .01367 * h   
    y = 1. - (x0**2)/6. + (n * x0**4 / 120.) - (n*(8*n -5)*x0**6/15120.)
    z = -(x0/3.) + (n*x0**3/30.) - ( 6*n*(8*n -5)*x0**5/15120. )
    
    if disp: print 'z0 = %f, y0 = %f' % (z,y)
       
    i = 0
    while y > 0:
        
        try:
        
            x1 =  x0 + h*i
            x23 = x1 + .5*h
            x4 = x1 + h
        
            k1 = h*z
            l1 = -h*( (y**n) + (2./x1)*z)
        
            z2 = z + (.5 * l1)
            k2 = h * z2
            l2 = -h * ( (y + .5 * k1)**n  + (2./x23) * z2  )
        
            z3 = z + (.5*l2)
            k3 = h * z3
            l3 = -h * (  (y + .5 * k2)**n  + (2./x23) * z3 )
        
            z4 = z + (.5*l3)
            k4 = h * (z + l3)
            l4 = -h * ( (y + k3)**n  + (2./x4) * z4 )
        
            dz = (l1 + 2*l2 + 2*l3 + l4) / 6.
            dy = (k1 + 2*k2 + 2*k3 + k4) / 6.        
        
        
            if (not(i%ndisp)) and disp:
                print'--------'
                print 'x1',x1
                print 'l',l1,l2,l3,l4
                print 'k',k1,k2,k3,k4
                print 'dy,dz',dy,dz
                print 'y,z',y,z
        
            z += dz
            y += dy
        
            i += 1
        
            _y.append(y)
            _x.append(x1)
            _z.append(z)    
    
            if disp: print 'x_1 = %f, y_1 = %f, z_1 = %f, D = %f' % (_x[-1],_y[-1],_z[-1], -_x[-1]**2*_z[-1])
            
        except:
            break
    
    return np.array(_x), np.array(_y), np.array(_z)

# some exact solutions, for comparison

def poly1(x):
    return np.sin(x)/x

def poly0(x):
    return 1 - x*x/6.

def poly5(x):
    return (1+x*x/3)**-.5


In [11]:
x1,y1,z1 = rk4(1.,1e-4)
x15,y15,z15 = rk4(1.5,1e-4)
x2,y2,z2 = rk4(2.,1e-4)
x3,y3,z3 = rk4(3.,1e-4)

In [29]:
l = 100

# downsample to make the plot cleaner
_x1,_y1 = x1[::l],y1[::l]
_x15,_y15 = x15[::l],y15[::l]
_x2,_y2 = x2[::l],y2[::l]
_x3,_y3 = x3[::l],y3[::l]

plot(_x1,_y1,'k--',label='n=1')
plot(_x15,_y15,'k+',label='n=1.5')
plot(_x2,_y2,'k.',color='r',label='n=2')
plot(_x3,_y3,'k:',color='g',label='n=3')

ylabel('$\\theta_n$')
xlabel('$\\xi$')
legend()

In [70]:
import pandas as pd

cols = ['n', ' \\( \\xi_1 \\)', '\\( \\frac{d\\theta_n (\\xi_1)}{d\\xi} \\)']
m = [[1,1.5,2,3],[x1[-1],x15[-1],x2[-1],x3[-1]],[z1[-1],z15[-1],z2[-1],z3[-1]]]
m = np.array(m).transpose()


m = pd.DataFrame(m,columns=cols)

In [71]:
print m.to_latex(escape=False,index=False)

\begin{tabular}{rrr}
\toprule
   n &   \( \xi_1 \) &  \( \frac{d\theta_n (\xi_1)}{d\xi} \) \\
\midrule
 1.0 &      3.141501 &                             -0.318306 \\
 1.5 &      3.653601 &                             -0.203305 \\
 2.0 &      4.352801 &                             -0.127246 \\
 3.0 &      6.896901 &                             -0.042428 \\
\bottomrule
\end{tabular}



In [62]:
m

,n,\( \xi_1 \),\( \frac{d\theta_n (\xi_1)}{d\xi} \)
0,1.0,3.140001,-0.318306
1,1.5,3.650001,-0.203306
2,2.0,4.350001,-0.127246
3,3.0,6.890001,-0.042428


## g. Eddington Mass

In [151]:
x,y,z = rk4(3,1e-4)
x1 = x[-1]
D = -x1**2.*z[-1]

In [67]:
# did this one in MKS out of habit
G = 6.674e-11
a = 7.5657e-16
m = 1.67372e-27
k = 1.3806e-23
m_s = 1.98855e30
m_edd = np.sqrt(  48*k**4 / (m**4 * G**3 * np.pi * a) ) * D_3

In [38]:
m_edd / m_s

17.99894694400281

## h. Solar parameters

In [3]:
x,y,z = rk4(3,1e-4)
x1 = x[-1]
D = -x1**2.*z[-1]

In [4]:
G = 6.674e-8 #cgs...
R = 6.957e10
M = 1.988e33
T=5e6
mu = 4./(.73*6+.25+2)

k = 1.38e-16 
s = 5.67e-5 
c = 3e10
a = 4*s/c
mH = 1.6737e-24

b = 1 - 4e-4

K =((3/a)**(1./3.)) * ((k/mH)**(4./3.)) * (18.**(-2./3.))

rho_c= ( np.sqrt((np.pi * G)/K) * (R/x1)  )**(-3.)

P_c = (rho_c**(4./3.)) * K

T_c = mu * b * mH *P_c / (rho_c * k)


In [5]:
rho_c

76.352293648772473

In [6]:
P_c / 1e15

124.37016980748963

In [7]:
K/1e14

3.8396070170794974

In [8]:
T_c / 1e6

11.914213531864329

## i. Chandrasekhar Mass

In [27]:
Z_A = .5
m_p = 1.67e-24
h = 6.626e-27
c=3e10
M_sun = 1.99e33
G = 6.674e-8

x,y,z = rk4(3,1e-4)
D = -x[-1]**2*z[-1]

In [28]:
K = (2 * np.pi * h * c) * (  3*Z_A   /  (8*np.pi*m_p)  )**(4./3.)
M = D * 4 * np.pi * (K/(np.pi*G))**(3./2.)

In [29]:
M/M_sun # Off, again

7.4835420520833251

# 3 Minimum mass for hydrogen fusion

## a

In [36]:
D = -x15[-1]**2. * z15[-1]
x_1 = x15[-1]
K = 1e13
G = 6.674e-8
M_sun = 1.99e33

#R = x_1*K*(5./2)*D/(4*np.pi * G)

R = (1./(4*np.pi * D))**(-1./3.) * (5*K/(8*np.pi*G)) * x_1

In [37]:
(R / (M_sun)**(1./3) ) / 1e9

2.8078970037965028

In [38]:
R

3.5318224723725812e+20

## b

In [32]:
R_sun = 69.57e9
m = .1 * M_sun
R0 = 2.8e9 * (10)**(1./3.)
R = .1 * R_sun

eta = 1e-9 * 5e6 * (10.)**(4./3.) * (R/R0)**2

In [33]:
eta

0.143273085884531

## d

In [31]:
T_c = 3e6
(T_c/ 2e8)**(3./4)

0.042861606445481995